<h3 align="center" style="font-size:35pt" > HW 2

    
<p style="font-size:20pt">
<u>Team Members: </u>  
<p style="font-size:18pt">
Brandt Lawson <br\><br\>  
insert here   <br\><br\>  
insert here   <br\><br\>  
insert here

# <p style="font-size:20pt; text-decoration: underline;"> Part 2: LTCM Risk  </p> 
<br/>

## <p style="font-size:18pt"> 1. Summary Stats
 
<p style="font-size:16pt"> a) For both the gross and net series of LTCM excess returns, report the mean and volatility. Since this is monthly data, scale the mean by 12, and scale the volatility by <font size="-1.5">$\sqrt{12}$ <font size="+3">

In [85]:
import pandas as pd
import numpy as np
import math

# ./ steps into this file's folder and lets you access the data file by just its name
path_to_data_file = './hedge_data.xls'
raw_data = pd.read_excel(path_to_data_file)

mean = raw_data.mean()*12
vol = raw_data.std()*math.sqrt(12)
df = pd.DataFrame(data={'mean': mean, 'vol': vol})

df
#market_return.iloc[:,1].values

,mean,vol
gross,0.257153,0.137987
net,0.168918,0.113187


<p style="font-size:16pt"> b) Report the annualized Sharpe ratio, (the SR based on the annualized mean and volatility.)


In [55]:
df["Sharpe"] = df['mean']/df['vol']
df


,mean,vol,Sharpe
gross,0.257153,0.137987,1.863603
net,0.168918,0.113187,1.492373


<p style="font-size:16pt"> c) Comment on whether the mean, volatility, and Sharpe ratio seem especially high or low relative to other assets we have seen.

* Commentary goes here

## 2. Using the series of net LTCM excess returns, denoted  $\tilde{r}$<sub>$t$</sub><sup>LTCM</sup>, estimate the following <p style="margin-left:1.25em">regression:</p> 
<h3 align="center" style="font-size:20pt">
    $\tilde{r}$<sub>$t$</sub><sup>LTCM</sup> =
    $\alpha$ +
    $\beta$<sup>m</sup> $\tilde{r}$<sub>$t$</sub><sup>m</sup> +
    $\epsilon$<sub>$t$</sub> <br/>

<p style="font-size:16pt"> a) Report $\alpha$ and $\beta$<sup>m</sup>  Report the $R$<sup>2</sup> stat.

In [147]:
from scipy import stats

market_return = pd.read_excel('./hedge_data.xls', sheet_name='MktExcessRets',  index_col='date')
#display(market_return)

beta_hat = market_return.loc['1994-04-30':'1998-06-30',:].values
#step2 = np.linalg.inv(beta_hat @ market_return.iloc[:,[1]].values)
#market_beta = (step2 * market_return.iloc[:,[1]].transpose().values)*raw_data['net']

x = beta_hat.reshape(51,)
y = raw_data.loc[:,'net'].values

slope, intercept, rvalue, pvalue, stderr = stats.linregress(x, y)

alpha = intercept
beta_m = slope
display('alpha = ' + str(alpha),'beta = ' + str(beta_m),'rsquared = ' + str(rvalue))

'alpha = 0.013430806247735787'

'beta = 0.042027927715985784'

'rsquared = 0.04063656913244539'

<p style="font-size:16pt"> b) From this regression, does LTCM appear to have much exposure to the equity-market factor,  $\tilde{r}$<sub>$t$</sub><sup>m</sup> ?

## 3. Regression-based metrics.


<p style="font-size:16pt">a) Calculate the Treynor ratio.



<p style="font-size:16pt">b) Calculate the Information ratio.



## 4. Tail risk.
<p style="font-size:16pt">a) Calculate the 5th worst return of the sample.

<p style="font-size:16pt">b) Calculate the mean of the worst 4 returns from the sample.

<p style="font-size:16pt"> c) Report the skewness of the return distribution. Compare to a normal distribution with <p style="margin-left:1.15em;font-size:16pt"> skewness of 0.</p>

<p style="font-size:16pt">d) Report the kurtosis of the return distribution. Compare to a normal distribution with
<p style="margin-left:1.15em;font-size:16pt"> kurtosis of three.</p>

# <p style="font-size:20pt"> <p style="text-decoration: underline;"> Part 3: Other Hedge Fund Indexes  </p> 
<br/>



## <p style="font-size:18pt"> 1. Report the summary statistics for each series:

# Sources

https://www.w3schools.com/html/html_styles.asp

https://developer.mozilla.org/en-US/docs/Web/HTML/Element/sup

https://csrgxtu.github.io/2015/03/20/Writing-Mathematic-Fomulars-in-Markdown/

https://www.w3schools.com/python/python_ml_linear_regression.asp